In [ ]:
graphics_toolkit("gnuplot"); % use if "plot" does not work

## Finite element method (FEM) in 2d
Solve the following 2d diffusion boundary value problem using linear finite elements.

In [ ]:
%% 2d boundary value problem:
%   -div(k(x)*grad(u(x)))=f(x) inside of the domain
%   u(x)=U(x) at GammaD
%   k(x)*(du/dn)(x)=g(x) at GammaN

Use prepared triangulation, i.e. prepared input matrices NODE, ELEM:

In [ ]:
%% example: load triangulation of a circular domain
load('network.mat');
[ NODE, ELEM, DBOUNDARY, NBOUNDARY ] = format_triangulation( p, e, t );
coords1=NODE(:,1); coords2=NODE(:,2);
n_NODE=size(NODE,1);
n_ELEM=size(ELEM,1);

Preparation of boundary conditions and material inputs:

In [ ]:
DVALUE=ones(length(DBOUNDARY),1);
NVALUE=-0.1*ones(size(NBOUNDARY,1),1);
FREENODE=true(n_NODE,1); FREENODE(DBOUNDARY)=false;

%% other input data
MATERIALS=ones(n_ELEM,1);
F=-0.5*ones(n_ELEM,1);

Construct FEM matrix and right hand side vector using local matrices and vectors:

In [ ]:
%% construction of global "stiffness" matrix and rhs
AREAS=polyarea(coords1(ELEM),coords2(ELEM),2);
A=zeros(n_NODE);
b=zeros(n_NODE,1);
for i=1:n_ELEM
    % add local "stiffness" matrix
    x=NODE(ELEM(i,:),:);
    B_=[x(2,2)-x(3,2) x(3,2)-x(1,2) x(1,2)-x(2,2)
        x(3,1)-x(2,1) x(1,1)-x(3,1) x(2,1)-x(1,1)];
    A_local=MATERIALS(i)*B_'*B_/4/AREAS(i);
    A(ELEM(i,:),ELEM(i,:))=A(ELEM(i,:),ELEM(i,:))+A_local;
    % add local rhs
    b_local=ones(3,1)/3*F(i)*AREAS(i);
    b(ELEM(i,:))=b(ELEM(i,:))+b_local;
end

Eliminate unknowns corresponding to Dirichlet nodes, modify the r. h. s. vector according to Neumann boundary conditions, and solve the resulting linear system:

In [ ]:
%% modifications due to boundary conditions
u=zeros(n_NODE,1);
u(DBOUNDARY)=DVALUE;
b=b-A*u;
for i=1:length(NVALUE)
    x=NODE(NBOUNDARY(i,:),:);
    b(NBOUNDARY(i,:))=b(NBOUNDARY(i,:))+norm(x(1,:)-x(2,:))*NVALUE(i)/2;
end

Plot solution:

In [ ]:
%% solution of the resulting linear system and visualization
u(FREENODE)=A(FREENODE,FREENODE)\b(FREENODE);
figure; trimesh(ELEM,coords1,coords2,u);